In [ ]:
"""
分析：
    通过train_news.tsv + test_news.tsv 将新闻分类，建立一个 Category -- Nid 的字典，News_Category
    通过train.tsv建立一个用户字典，即 Uid--Nid(Category)的字典，表示用户感兴趣的新闻的种类， User_Category
    
    对test.tsv，对于每个要预测的用户，
        （1）如果history的新闻id在 则根据User_Category,
            若用户对此新闻的类别有点击历史，则直接添加“-1”
    
"""


In [2]:
 #导入必要的函数库 
import numpy
import random 
import pandas as pd 
import math
from sklearn.model_selection import train_test_split
from sklearn import datasets
from collections import defaultdict

In [4]:
train_news_path = r"./train/train_news.tsv"
train_history_path = r"./train/train.tsv"

In [5]:
train_history = pd.read_csv(train_history_path, sep='\t',header=0)
train_history

,Uid,Date,History,Impression
0,U48196,11/11/2019 7:56:24 PM,N51591 N23288 N33604 N3908 N16730 N23571 N1673...,N58410-0 N53696-0 N51797-0 N64482-0 N4936-0 N1...
1,U46641,11/14/2019 1:03:08 PM,N44598 N48076 N31248 N11673 N18106 N3128,N20270-0 N34185-0 N38779-1 N45523-0
2,U48973,11/14/2019 9:47:23 AM,N27612 N24569 N51706 N63526 N62058,N23446-0 N12446-0 N58660-0 N38779-0 N6477-0 N3...
3,U15980,11/12/2019 6:14:10 AM,N52518 N16636 N32223 N60485 N43725 N9460 N3654...,N57713-0 N25722-0 N20043-0 N20041-0 N57459-0 N...
4,U48830,11/10/2019 1:42:02 PM,N1570 N9808 N39813 N10059 N30708 N15771 N22469...,N53245-0 N4936-0 N27101-0 N64513-0 N15224-0 N3...
...,...,...,...,...
9995,U38291,11/11/2019 7:39:38 AM,N64049 N54827 N14801 N42620 N47277 N52551,N59981-0 N55689-0 N42961-0 N13801-1 N154-0 N63...
9996,U80022,11/11/2019 4:25:51 PM,N11264 N62931 N37327 N38629 N29177 N56586 N624...,N16920-0 N51825-0 N60872-0 N28837-0 N42597-0 N...
9997,U75259,11/9/2019 7:52:43 PM,N49944 N19693 N22561 N19495,N45211-0 N36752-0 N59852-0 N48958-0 N1034-0 N4...
9998,U44699,11/10/2019 8:14:43 AM,N25691 N53636 N29177 N51112 N61471 N62287 N191...,N30836-0 N47453-0 N26130-0 N350-0 N39471-0 N57...


In [23]:
train_news = pd.read_csv(train_news_path, sep='\t',header=0)
train_news.

,Nid,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...,...,...,...
51277,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51278,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN
51279,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."
51280,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ..."


In [7]:
#处理新闻

In [30]:
n = open(train_news_path,"rt",encoding="utf-8")
header = n.readline()
header = header.strip().split('\t')#列名
print(header)

['Nid', 'Category', 'SubCategory', 'Title', 'Abstract']


In [35]:
news_dataset = []

for line in n:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    news_dataset.append(d)

In [41]:
news_dataset[0]["Nid"]

'N55528'

In [42]:
News_Category = defaultdict(set)

for d in news_dataset:
    nid, category = d['Nid'],d["Category"]
    News_Category[category].add(nid)

In [45]:
News_Category

defaultdict(set,
            {'lifestyle': {'N43529',
              'N49618',
              'N24264',
              'N48776',
              'N46337',
              'N19032',
              'N18001',
              'N25295',
              'N63823',
              'N22702',
              'N43484',
              'N3937',
              'N48941',
              'N3240',
              'N19317',
              'N35500',
              'N56005',
              'N38471',
              'N23477',
              'N3122',
              'N9513',
              'N39817',
              'N35002',
              'N11612',
              'N62480',
              'N9056',
              'N1936',
              'N47433',
              'N35541',
              'N8555',
              'N65090',
              'N30865',
              'N51213',
              'N2581',
              'N64829',
              'N7837',
              'N827',
              'N61471',
              'N40899',
              'N33276',
              'N37717

In [ ]:
#处理用户

In [50]:
u = open(train_history_path,"rt",encoding="utf-8")
header2 = u.readline()
header2 = header2.strip().split('\t')#列名
print(header2)

['Uid', 'Date', 'History', 'Impression']


In [51]:
users_dataset = []

for line in u:
    fields2 = line.strip().split('\t')
    u = dict(zip(header2, fields2))
    users_dataset.append(u)

In [52]:
users_dataset[0]

{'Uid': 'U48196',
 'Date': '11/11/2019 7:56:24 PM',
 'History': 'N51591 N23288 N33604 N3908 N16730 N23571 N16730 N44018 N5844 N23979 N48369 N37706 N15745 N28487 N43353 N54529 N23488 N3682 N26318 N18896 N6004 N11701 N10503 N29088 N719 N37264 N54069 N30026 N871 N27167 N37378 N6233 N51661 N33115 N29101 N26978 N24920 N59754 N31322 N20039 N13603 N33200 N62705 N4394 N3680 N50513 N11175 N1088 N47525 N35820 N41351 N1603 N34198 N49229 N31931 N59763 N37127 N22081 N63376 N56693 N49458 N54827 N274 N9803 N25905 N26272 N3613 N59359 N48259 N21178 N51541 N24492 N26503 N29742 N55743 N2086 N28679 N38299 N10848 N3130 N41997 N23045 N7851 N19280 N57760 N17542 N60360 N64609 N5351 N34000 N43142 N58391 N33742 N7853 N14385 N39161 N47482 N26122 N48356 N11231 N53422 N7019 N26826 N17050 N14761 N25258 N24409 N28856 N61355 N39414 N16922 N15663 N10814 N6785 N55001 N32004 N62211 N25912 N13753 N54072 N7646 N55055 N6767 N31281 N22012 N22816 N15676 N41881 N34520 N1294',
 'Impression': 'N58410-0 N53696-0 N51797-0 N64482-

In [53]:
User_list = defaultdict(set)

for d in users_dataset:
    uid, history  = d["Uid"],d['History']
    User_list[uid].add(history)

In [54]:
User_list

defaultdict(set,
            {'U48196': {'N51591 N23288 N33604 N3908 N16730 N23571 N16730 N44018 N5844 N23979 N48369 N37706 N15745 N28487 N43353 N54529 N23488 N3682 N26318 N18896 N6004 N11701 N10503 N29088 N719 N37264 N54069 N30026 N871 N27167 N37378 N6233 N51661 N33115 N29101 N26978 N24920 N59754 N31322 N20039 N13603 N33200 N62705 N4394 N3680 N50513 N11175 N1088 N47525 N35820 N41351 N1603 N34198 N49229 N31931 N59763 N37127 N22081 N63376 N56693 N49458 N54827 N274 N9803 N25905 N26272 N3613 N59359 N48259 N21178 N51541 N24492 N26503 N29742 N55743 N2086 N28679 N38299 N10848 N3130 N41997 N23045 N7851 N19280 N57760 N17542 N60360 N64609 N5351 N34000 N43142 N58391 N33742 N7853 N14385 N39161 N47482 N26122 N48356 N11231 N53422 N7019 N26826 N17050 N14761 N25258 N24409 N28856 N61355 N39414 N16922 N15663 N10814 N6785 N55001 N32004 N62211 N25912 N13753 N54072 N7646 N55055 N6767 N31281 N22012 N22816 N15676 N41881 N34520 N1294'},
             'U46641': {'N44598 N48076 N31248 N11673 N18106 N3128'},
   